In [224]:
# Import libraries
import urllib.request
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import re
from operator import add
import xlrd
import math

In [225]:
# Get company list
companies = pd.read_csv(r'C:\Users\Harry Shuttleworth\Desktop\Python\Financial scraper\US_companies.csv')
company_name = str(companies['Name'][3])
company_cik = str(companies['CIK code'][3])

In [226]:
# Get document link and download
# Get link to interactive data
company_href="https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=" + company_cik\
 + "&type=10-k&dateb=&owner=exclude&count=40"
response1 = requests.get(company_href).content
soup1 = BeautifulSoup(response1, 'html.parser')
Interactive_text = []
for a in soup1.find_all('a', href=True): 
    if a.text.find("Interactive Data") == True:
        Interactive_text.append(a['href'])
        
data_href = "https://www.sec.gov/" + Interactive_text[0]        

# Get link to Excel document
response2 = requests.get(data_href).content
soup2 = BeautifulSoup(response2, 'html.parser')
for a in soup2.find_all('a', href=True): 
    if a.text == "View Excel Document":
        link_text = a['href']
        
# Download document
href = "https://www.sec.gov/" + link_text
print('Downloaded file with urllib:')
urllib.request.urlretrieve(href, "C:/Users/Harry Shuttleworth/Desktop/Python/US_data.xlsx")

Downloaded file with urllib:


('C:/Users/Harry Shuttleworth/Desktop/Python/US_data.xlsx',
 <http.client.HTTPMessage at 0x17d8d0bf108>)

In [227]:
# Create dataframe from Excel and find correct worksheet
xls = xlrd.open_workbook(r'C:\Users\Harry Shuttleworth\Desktop\Python\US_data.xlsx', on_demand=True)
sheet_names = xls.sheet_names()
sheet_names
for sheet in sheet_names:
    lower_sheet = str.lower(sheet)
    if lower_sheet.find("consolidated") > -1 and lower_sheet.find("statement") > -1:
        sname = sheet
        break
    else:
        continue
df = pd.read_excel(r'C:\Users\Harry Shuttleworth\Desktop\Python\US_data.xlsx', sheet_name = sname)

In [228]:
# Reorganise columns and rows
columns = df.columns.tolist()
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header

for i in range(len(columns)):
    if i == 0:
        continue
    else:
        if columns[i].find("Unnamed") >= 0:
            columns[i] = last
    last = columns[i]

In [229]:
# Get currency code
position = columns[0].find("-")
currency = columns[0][position + 2: position + 5]

# Get units for shares and money
#shares = re.search(r'\bshares in\s+\w+', columns[0]).group() #Might be useful in future
units = re.findall(r'\bin\s+\w+', columns[0])
units[0] = units[0][3:]
units[1] = units[1][3:]

shares = units[0]
money = units[1]

if len(units) > 2:
    print("Error, table formatting has changed.")

In [240]:
# Amend column headers with time periods
columns[0] = "Metric"
cols = df.columns.tolist()
cols[0] = ""
cols = list(map(str, cols))
df.columns = df.columns.fillna('to_drop')

# Remove extra nan columns if they exist
nan_index = [i for i,x in enumerate(cols) if str(x) == 'nan']
df.drop(df.columns[nan_index], axis = 1, inplace=True)
cols = df.columns.tolist()
#drop index from columns list
for nan in nan_index:
    columns.pop(nan)

headers = [i +"-"+ j for i, j in zip(columns, cols)]
df.columns = headers
df.rename(columns={"Metric-": "Metric"}, inplace=True)

# to_drop = [i for i,x in enumerate(df.columns.tolist()) if x.find("to_drop") > -1]
# df.drop(df.columns[to_drop], axis = 1, inplace=True)

# Remove duplicate rows
df.drop_duplicates(subset ="Metric", keep = 'first', inplace = True) 

KeyError: Index(['Metric'], dtype='object')

In [239]:
df

,"Metric-Dec. 31, 2018","12 Months Ended-Dec. 31, 2017","12 Months Ended-Dec. 31, 2016"
1,NaN,NaN,NaN
2,3440,3357,3302
3,NaN,NaN,NaN
4,1479,1369,1499
5,545,492,470
6,277,259,258
7,-20,-16,-10
8,57,0,0
9,2338,2104,2217
10,1102,1253,1085


In [ ]:
# Reshape dataframe so data moves from wide to long format
df.set_index('Metric', inplace=True)
stacked_df = df.stack()
df = stacked_df.reset_index()

# Rename columns, add company name column, and period column
df.rename(columns={'level_1': 'Period_End', 0: 'Value'}, inplace=True)
df['Company name'] = company_name
col_str = df.Period_End.str.split("-")
df['Period'] = [x[0] for x in col_str]
df['Period_End'] = [x[1] for x in col_str]
df['Period'] = df['Period'].str.replace(" Ended", "")
df['Currency'] = currency
df['Units'] = money
df['Shares'] = shares

In [ ]:
df